In [208]:
from typing import List
from functools import lru_cache, reduce
from itertools import chain, starmap, accumulate, takewhile

import arrays
from importlib import reload
reload(arrays)
import arrays as ar


In [218]:
def max_histogram_rectangle(h: list) -> int:
    s = []
    def it(j, v):
        while s and h[s[-1]] > v:
            yield h[s.pop()] * ((j - s[-1] -1) if s else j)
        s.append(j)
    rect = lambda jv: reduce(max, it(*jv), 0)
    return max(max(map(rect, enumerate(h))), rect((len(h), 0)))


In [219]:
max_histogram_rectangle([8, 6])


12